# Arrests

Initially from Franzen scraped.

In [1]:
# Libraries, base baths, etc
import pandas as pd
import numpy as np
import json

dirData = '../data/'
dirDataExt = dirData + 'external/'
dirDataProc = dirData + 'processed/'

dirScrapeFran = dirDataExt + 'franzen-scrape/'

In [2]:
# Helper to read the ndljson's
def readNDLJson(f):
    with open(f) as data:
        jsonList = [json.loads(line) for line in data]
    return jsonList

# Helper function to format the arrests
def formatRecord(record, ind=False):
    if (ind == True):
        record['datetime'] = record['datetime']['$date']
    record['_id'] = record['_id']['$oid']
    
    return record

In [3]:
# Read and format the data

actJson = readNDLJson(dirScrapeFran + 'activities.json')
for arrest in actJson:
    arrest = formatRecord(arrest, True)
    
actListJson = readNDLJson(dirScrapeFran + 'activityList.json')
for activity in actListJson:
    activity = formatRecord(activity)

disJson = readNDLJson(dirScrapeFran + 'dispositionList.json')
for disp in disJson:
    disp = formatRecord(disp)

In [4]:
actJson[2020]

{'_id': '50fcbd7ec0025f8c936c11ae',
 'activity': 'ANIMAL/NEGLECT',
 'address': '938 Longfellow Place, Iowa City, IA 52240, USA',
 'apt': '',
 'date': {'$date': '2013-01-07T14:29:00.000+0000'},
 'datetime': '2013-01-07T14:29:00.000+0000',
 'details': 'Linked to CFS#: 13003060',
 'dispatch': '13003059',
 'disposition': 'COMPLETED',
 'inc': '00234',
 'link': '?dis=13003059&date=01072013',
 'loc': [-91.515237, 41.650727],
 'time': '02:29 pm'}

In [16]:
actExcCol = ['_id', 'address', 'addresses', 'apt', 'date', 'dispatch', 'inc','lat', 'lon', 'link', 'time']
test = pd.DataFrame.from_records(actJson, exclude=actExcCol)

In [17]:
test.iloc[2500:2600]

,activity,datetime,details,disposition,loc
2500,ESCORT/RELAY,2013-01-04T13:50:00.000+0000,CAR 57,COMPLETED,"[-91.5401082, 41.6466355]"
2501,ANIMAL/K9 TRAINING,2013-01-04T13:49:00.000+0000,None,COMPLETED,"[-91.529432, 41.661081]"
2502,OUT FOR INVEST/FOLLOW UP,2013-01-04T12:43:00.000+0000,None,COMPLETED,"[-91.5770117, 41.6562309]"
2503,INFO/ALL OTHER INFO,2013-01-04T12:41:00.000+0000,"WILL BE TESTING PANIC ALARM, DO NOT NEED ANYTH...",COMPLETED,"[-91.510495, 41.647059]"
2504,SUSPICIOUS/ACT/PERS/VEH,2013-01-04T12:29:00.000+0000,MALE SUBJECT OUTSIDE HIS BUSINESS HAS A KNIFE ...,CHARGED/RELEASED,"[-91.531137, 41.6534819]"
2505,TR/PARKING,2013-01-04T12:15:00.000+0000,SPACE 61 A HONDA CIVIC TO TOW,PARKING TICKET,"[-91.5346163, 41.6579084]"
2506,SUICIDE/LAW,2013-01-04T12:14:00.000+0000,"FEMALE IS SUICIDAL/IS IN THE OFFICE, CALLER IS...",COMPLETED,"[-91.528454, 41.658617]"
2507,TR/PARKING,2013-01-04T12:05:00.000+0000,HERE AT FRONT DESK- JUST PURCHASED THE BLDG. C...,VEHICLE TOWED,"[-91.540505, 41.6500554]"
2508,TR/PARKING,2013-01-04T11:57:00.000+0000,CAR ACROSS FROM DRIVEWAY FOR ABOUT TWO WEEKS W...,COMPLETED,"[-91.517483, 41.642354]"
2509,PUBLIC ASSIST,2013-01-04T11:55:00.000+0000,VIN VERIFICATION CHECK BLK JEEP COMPASS PA...,COMPLETED,"[-91.529432, 41.661081]"
